# Computer Simulation Project Documentation
### Full Names : MohammadDanial Jahanbani, Erfan Sadrayie
Student Number : 99105367, 99101835
___
## Libraries
We used `Simpy` to simulate the 1-core system as a resource in our code and finally used `Pandas` to display the data for tasks.
## Implementation
based on the system description we implemented the below algorithm. It consists of three main parts:
</br>
</br>
<li>
    <b>Creating jobs</b>: with time intervals $t_i$ exponentially distributed with rate $X$, jobs are created with a service time also exponentially distributed with rate $Y$ plus an exponentially distributed renege time with parameter $Z$. In addition, a priority is generated for this job which equals:$$pq\_priority = creation\_time \times c + priority$$ where $c$ indicates the imporance of creation time in proportion to the actual priority i.e 1 for high, 2 for normal and 3 for low.
</li>
</br>
<li>
    <b>Transferring jobs from layer 1 to Round Robin 1 </b>: In an iterative fashion, after each $T$ seconds the total number of tasks in second layer queues is checked and if it is less than $k$, the first $k$ elements are added to <i>Round Robin 1</i> priority queue.
</li>
</br>
<li>
    <b>Transferring jobs from a queue in layer 2 to the CPU core</b>: Based on our queue selection policy, we can operate two different queue selection for the core. if the policy is <i>Normal</i> then  at First the core will perform tasks from the <i>Round Robin 1</i> priority queue and if that is empty then it will accept tasks from <i>Round Robin 2</i> Priority queue and finally <i>FCFS</i> priority queue. However, if the policy is <i>Random</i> then the core will be provided with a task randomly selected from one of the three queues in layer 2.
</li>
</br>
<li>
    <b>Transferring jobs between queues of layer 2</b>: If a job in layer 2 has a service time higher than the quantum time of the queue it is currently in, then it should be moved to a queue with lower priority after it has been given service by the core equal to the quantum time of the queue it is currently in. 
</li>
</br>
<li>
    <b>Checking for reneged jobs</b>: We iteratively check to see if any jobs have timed out and remove them from the queue they are currently in. If a job is being served by the core, it will not be affected by this. 
</li>

### Average Queue Lengths for different rates
the below code does the job $X$, $Y$, $Z$:

In [1]:
from main import simulate
from numpy import mean

results = []
for x in range(5, 15, 2):
    for y in range(5, 15, 2):
        for z in range(0, 1000, 100):
            jobs_result, queue_result = simulate(x, y, z)
            print('entrance rate:', x, 'service rate:', y)
            print('average PQ length:', mean(queue_result['PQ length']))
            print('average R1 length:', mean(queue_result['R1 length']))
            print('average R2 length', mean(queue_result['R2 length']))
            print('average FCFS length', mean(queue_result['FCFS length']))

entrance rate: 5 service rate: 5
average PQ length: 0.00575
average R1 length: 0.0
average R2 length 0.0
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 0.79625
average R1 length: 0.17825
average R2 length 0.0255
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 1.4505
average R1 length: 0.25525
average R2 length 0.05375
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 2.30325
average R1 length: 0.62
average R2 length 0.16825
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 2.2825
average R1 length: 1.12425
average R2 length 0.18425
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 4.5225
average R1 length: 1.0745
average R2 length 0.1085
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 4.01575
average R1 length: 0.7105
average R2 length 0.03625
average FCFS length 0.0
entrance rate: 5 service rate: 5
average PQ length: 4.80975

### average time spent in queue per job and per queue
we can display the table for average waiting times in one of the queues:

In [2]:
from pandas import DataFrame
x = 20
y = 30
z = 200
jobs_result, queue_result = simulate(x, y, z)
print(jobs_result)
print('average time in PQ:', DataFrame.mean(jobs_result['PQ wt']))
print('average time in R1:', DataFrame.mean(jobs_result['R1 wt']))
print('average time in R2:', DataFrame.mean(jobs_result['R2 wt']))
print('average time in FCFS:', DataFrame.mean(jobs_result['FCFS wt']))

    id  priority  service time  created time  is done  PQ wt         R1 wt  \
0    1         2             1             0     True   90.0  9.300000e+01   
1    2         3             3            10     True   80.0  9.710000e+01   
2    3         2             3            28     True  362.0  3.800000e+00   
3    4         3            42            42    False   18.0  7.800000e+01   
4    5         3             2            55    False  125.0  1.921000e+02   
5    6         3             1            82     True  128.0  2.202000e+02   
6    7         3             4            83     True  157.0  2.463000e+02   
7    8         3             2           100     True  260.0  3.668000e+02   
8    9         3            22           124    False   26.0           NaN   
9   10         2             3           126     True  714.0  2.136000e+02   
10  11         3            21           189    False   81.0  2.653000e+02   
11  12         3            45           190    False    NaN    

Notice that due to a relatively short simulation time, most jobs won't reach the last queue.
### expired jobs rate
We can also determine the portion of reneged jobs:

In [3]:
len(jobs_result[jobs_result['is done'] == False]) / len(jobs_result)

0.43333333333333335

### CPU utility
since the 1-core is always working: $$\rho=1$$

### how to improve average time spent in queues?
The quantum time of queues must be greater or equal to the average of job service times. This is concluded from the fact that if we let the quantum time be less than the average service job, since our structure will transfer most of the jobs to the next queue, the system could get unstable. since all the jobs will be accumulated on the last queue and either the jobs will renege or they will have a really long waiting time.